# LSTM: Air Passengers

based on https://github.com/mikel-brostrom/flight-passengers-prediction-LSTM/blob/master/flight_passengers_prediction_LSTM.ipynb

This example features training a LSTM model on the air passengers dataset.

In [ ]:
import auto_compyute as ac
import auto_compyute.nn.functional as F

ac.set_random_seed(0)

## Prepare Data

In [ ]:
import pandas as pd

# download the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv"
data = pd.read_csv(url)
tensor = ac.tensor(data.iloc[:, 1].values).float()

In [ ]:
def min_max_scaler(x, low, high, dim=None):
    x_min, x_max = x.min(dim), x.max(dim)
    return (x - x_min) * (high - low) / (x_max - x_min) + low

In [ ]:
# scale values
scaled_tensor = min_max_scaler(tensor, -1, 1)

# save values for inverse transformation
train_max = tensor.max().item()
train_min = tensor.min().item()

# apply a sliding window
lookback = 4
n_samples = len(scaled_tensor) - lookback
windowed_tensor = ac.stack(*[scaled_tensor[i : i + lookback + 1] for i in range(n_samples)])

x = windowed_tensor[:, :-1, None]
y = windowed_tensor[:, -1, None]

# create train and test splits
train_size = int(len(y) * 0.4)
X_train = x[:train_size]
y_train = y[:train_size]

print(f"{x.shape=}")
print(f"{y.shape=}")
print(f"{X_train.shape=}")
print(f"{y_train.shape=}")

## Build the Neural Network

In [ ]:
from auto_compyute import nn

# B = batch size

model = nn.Sequential(
    nn.LSTM(1, 2, return_seq=False),  # out: (B, 2)
    nn.Linear(2, 1),  # out: (B, 1)
)

## Training

In [ ]:
epochs = 1000
optim = nn.optimizers.Adam(model.parameters())

model.train()
for e in range(1, epochs + 1):
    optim.reset_param_grads()
    F.mse_loss(model(X_train), y_train).backward()
    optim.update_params()

## Evaluate Model

In [ ]:
import matplotlib.pyplot as plt

model.eval()
with ac.no_autograd_tracking():
    y_pred = model(x)
    y_pred_scaled = min_max_scaler(
        y_pred, low=train_min, high=train_max
    )  # rescale outputs to the original scale
    y_scaled = min_max_scaler(
        y, low=train_min, high=train_max
    )  # rescale outputs to the original scale

plt.axvline(x=train_size, c="r", linestyle="--", label="right limit of training data used")
plt.plot(y_scaled.data, label="train data")
plt.plot(y_pred_scaled.data, label="predictions")
plt.suptitle("Time-Series Prediction")
plt.legend()
plt.show()